# WeatherPy

---

## Starter Code to Generate Random Geographic Coordinates and a List of Cities

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Impor the OpenWeatherMap API key
from api_keys import weather_api_key
from api_keys import geoapify_api_key

# Import citipy to determine the cities based on latitude and longitude
from citipy import citipy

### Generate the Cities List by Using the `citipy` Library

In [2]:
# Empty list for holding the latitude and longitude combinations
lat_lngs = []

# Empty list for holding the cities names
cities = []

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
print(f"Number of cities in the list: {len(cities)}")

Number of cities in the list: 582


---

## Requirement 1: Create Plots to Showcase the Relationship Between Weather Variables and Latitude

### Use the OpenWeatherMap API to retrieve weather data from the cities list generated in the started code

In [4]:
# set base url
baseweatherUrl = "http://api.openweathermap.org/data/2.5/"
basegeoUrl = "https://api.geoapify.com/v2/"

weatherKey = weather_api_key
geoKey = geoapify_api_key

# def getGeoData(city, country, apikey):
#     endpoint = f"https://api.geoapify.com/v1/geocode/search?apiKey={apikey}&format=json&text={city}_{country}"
#     return endpoint

def getCityUrl(city, apikey):
    result = f"https://api.geoapify.com/v1/geocode/search?apiKey={apikey}&format=json&text={city}&type=city"
    return result

def getWeatherUrl(lat, lon, apikey):
    # result = f"{baseweatherUrl}/weather?appid={apikey}&lat={lat}&lon={lon}&units=imperial"
    result = f"{baseweatherUrl}weather?appid={apikey}&units=imperial&q={city}"
    return result

def getCityWeatherUrl(city, apikey):
    result = f"{baseweatherUrl}weather?appid={apikey}&units=imperial&q={city}"
    return result


### Test Code

In [28]:
# test code
testurl = getCityUrl("college",geoKey)
testurl = getCityWeatherUrl("college", weatherKey)
# testurl = getWeatherUrl("64.8569444","-147.8027778",weatherKey)
# resp = requests.get(testurl).json()["results"][0]
# lat = resp["lat"]
# lon = resp["lon"]
# print(f"Lat: {lat}, Lon: {lon}")


'http://api.openweathermap.org/data/2.5/weather?appid=ccf7ac95d3227a8e645528f05592bb2a&units=imperial&q=college'

### City Data

In [5]:
# Define an empty list to fetch the weather data for each city
city_data = []

# Print to output
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters
record_count = 1
set_count = 1

# Loop through all the cities in our list to fetch weather data
for i, city in enumerate(cities):
        
    # Group cities in sets of 50 for logging purposes
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 0

    # Create endpoint URL with each city
    city_url = getCityUrl(city,geoKey)
    
    # Log the url, record, and set numbers
    print("Processing Record %s of Set %s | %s" % (record_count, set_count, city))

    # Add 1 to the record count
    record_count += 1

    # Run an API request for each of the cities
    try:
        # Parse the JSON and retrieve data
        cityData = requests.get(city_url).json()
        lat = cityData["results"][0]["lat"]
        lon = cityData["results"][0]["lon"]

        # get weather data
        # weatherUrl = getWeatherUrl(lat, lon, weatherKey)
        weatherUrl = getCityWeatherUrl(city, weatherKey)
        weatherData = requests.get(weatherUrl).json()

        city_weather = weatherData["main"]["temp"]

        # Parse out latitude, longitude, max temp, humidity, cloudiness, wind speed, country, and date
        city_lat = lat
        city_lng = lon
        city_max_temp = weatherData["main"]["temp_max"]
        city_humidity = weatherData["main"]["humidity"]
        city_clouds = weatherData["clouds"]["all"]
        city_wind = weatherData["wind"]["speed"]
        city_country = cityData["results"][0]["country"]
        city_date = cityData["dt"]

        # Append the City information into city_data list
        city_data.append({"City": city, 
                          "Lat": city_lat, 
                          "Lng": city_lng, 
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

    # If an error is experienced, skip the city
    except:
        print(f"City '{city}' not found. Skipping...")
        pass
              
# Indicate that Data Loading is complete 
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | kalianget
City 'kalianget' not found. Skipping...
Processing Record 2 of Set 1 | ishpeming
City 'ishpeming' not found. Skipping...
Processing Record 3 of Set 1 | miracema do tocantins
City 'miracema do tocantins' not found. Skipping...
Processing Record 4 of Set 1 | aykhal
City 'aykhal' not found. Skipping...
Processing Record 5 of Set 1 | adamstown
City 'adamstown' not found. Skipping...
Processing Record 6 of Set 1 | isangel
City 'isangel' not found. Skipping...
Processing Record 7 of Set 1 | port-aux-francais
City 'port-aux-francais' not found. Skipping...
Processing Record 8 of Set 1 | hithadhoo
City 'hithadhoo' not found. Skipping...
Processing Record 9 of Set 1 | waitangi
City 'waitangi' not found. Skipping...
Processing Record 10 of Set 1 | ancud
City 'ancud' not found. Skipping...
Processing Record 11 of Set 1 | udaipura
City 'udaipura' not found. Skipping...
Processing Record 12 of Set 1

In [ ]:
weatherBaseUrl = f"{baseweatherUrl}weather?appid={weatherKey}"

url = f"{weatherBaseUrl}&q=Corona"

response = requests.get(url).json()

# url = "https://api.openweathermap.org/data/2.5/weather?appid=ccf7ac95d3227a8e645528f05592bb2a&q=Irvine"

response

In [11]:
placesBaseUrl = f"{basegeoUrl}places?apiKey={geoKey}"

url = getGeoData("Irvine","California", geoKey) # f"{placesBaseUrl}&categories=commercial"

url

'https://api.geoapify.com/v1/geocode/search?apiKey=59627f8b9e9a4c24931c6dbde5be5529&format=json&text=Irvine_California'